In [1]:
import os
from os import listdir
from os.path import isfile, join

import pandas as pd
from itertools import islice
import numpy as np

from skimage.external import tifffile
from skimage.external.tifffile import imread

import matplotlib.pyplot as plt

import ipywidgets as widgets

import random

In [2]:
bigDir = r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\DATA'
cellDataFile=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\sel_cells_20_50_10_box.csv'

het_percentile=10

In [3]:
cellData=pd.read_csv(cellDataFile)
cellData.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,file,original cell name,Amy_mod,movie,x,y,ab1 (heterochromatin),...,random_10perc_ch1,hetChrom_10perc_ch2,random_10perc_ch2,hetChrom_10perc_ch3,random_10perc_ch3,exp_folder,box_mask_volume,box_mask_nuc,box_mask_het,box_mask_nuc_box
0,0,0,0,20190625_cell-00.czi,7LE1(B),1130/10,190619-cdc6-dhb-pcna002xy09,172,956,HP1B,...,281528813.0,174067475.0,131640512.0,1.352231e+09,1.043517e+09,20190625-MSL rename,7200000.0,NaN,1921080.0,73920.0
1,1,1,1,20190625_cell-01.czi,7L-E1(T),0,190619-cdc6-dhb-pcna002xy09,158,906,HP1B,...,266516067.0,162411790.0,128299406.0,1.353023e+09,1.065184e+09,20190625-MSL rename,6720000.0,NaN,2446360.0,79002.0
2,2,2,2,20190625_cell-02.czi,7L-E2(B),0,190619-cdc6-dhb-pcna002xy09,832,252,HP1B,...,318905411.0,194622718.0,148176740.0,1.366670e+09,1.118365e+09,20190625-MSL rename,8160000.0,NaN,2783240.0,15318.0
3,3,3,3,20190625_cell-03.czi,7L-E2(T),0,190619-cdc6-dhb-pcna002xy09,888,160,HP1B,...,285348983.0,195376713.0,139563277.0,1.298882e+09,1.038559e+09,20190625-MSL rename,6560000.0,NaN,2240000.0,7310.0
4,4,4,4,20190625_cell-05.czi,7L-G2,0,190619-cdc6-dhb-pcna002xy09,172,324,HP1B,...,698512214.0,248463390.0,183210463.0,2.583220e+09,2.158853e+09,20190625-MSL rename,7040000.0,NaN,2204448.0,104400.0


In [20]:
cellData['hetChrom_newctl_volume']=0
cellData['hetChrom_newctl_ch1']=0
cellData['hetChrom_newctl_ch2']=0
cellData['hetChrom_newctl_ch3']=0

In [14]:
cellData=cellData.drop(columns=['box_mask_volume', 'box_mask_nuc', 'box_mask_het', 'box_mask_nuc_box'])

In [29]:
i=0

# Progress Bar
progBar=widgets.IntProgress(
    value=i,
    min=0,
    max=len(cellData),
    step=1,
    description='Progress:',
    orientation='horizontal'
)
display(progBar)

for i,myCell in islice(cellData.iterrows(), i,None):
    print(cellData.at[i,'file'])
    
    # open tiff image of a cell
    myImage=imread(myCell.path)    
    myChannel = myImage[:, 0, :, :] #1st channel is hetChrom (as stained by HP1B or H3K9Me3)

    # open nucleus mask
    nucleusMaskPath=myCell.path
    nucleusMaskPath=nucleusMaskPath.replace('data_tiff','segmentation_nucleus_Otsu')
    nucleusMaskPath=nucleusMaskPath.replace('.tif','_nucleus.tif')
    nucleusMask=imread(nucleusMaskPath)
    nucleusMask=nucleusMask.astype(bool)

    # open het mask - new control
    myHetMaskPath=myCell.path
    myHetMaskPath=myHetMaskPath.replace('data_tiff','mask_box_method')
    myHetMaskPath=myHetMaskPath.replace('.tif','_box_mask.tif')
    try:
        myHetMask=imread(myHetMaskPath)
        myHetMask=myHetMask.astype(bool)
        # calculate signals
        df = pd.DataFrame(columns = ['HetChrom_Volume','HetChrom_newctl_ch1','HetChrom_newctl_ch2','HetChrom_newctl_ch3'])

        df.Slice = range(myChannel.shape[0])
        df.HetChrom_Volume =  myHetMask.sum(-1).sum(-1)
        #df.random_Volume =  scrMask.sum(-1).sum(-1)

        for ch in range(3):

            # percentile
            myChannel = np.copy(myImage[:, ch, :, :])
            myChannel[myHetMask] = 0
            signal = myChannel.sum(-1).sum(-1)
            exec(f'df.HetChrom_newctl_ch{ch+1}=signal')


        # save signals
        dfSavePath=myCell.path
        dfSavePath=dfSavePath.replace('data_tiff','results')
        dfSavePath=dfSavePath.replace('.tif',f'_hetChrom_newctl.csv')
        df.to_csv(dfSavePath)

        # add info to cell info file
        cellData.loc[i,f'hetChrom_newctl_volume']=np.sum(df.HetChrom_Volume)

        for ch in range(3):
            exec(f"cellData.loc[i,'hetChrom_newctl_ch{ch+1}']=np.sum(df.HetChrom_newctl_ch{ch+1})") 

    except:
        for ch in range(3):
            exec(f'df.HetChrom_total_newctl_ch{ch+1}=None')
    
    

    progBar.value=progBar.value+1

IntProgress(value=0, description='Progress:', max=300)

20190625_cell-00.czi


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


20190625_cell-01.czi
20190625_cell-02.czi
20190625_cell-03.czi
20190625_cell-05.czi
20190625_cell-06.czi
20190625_cell-07.czi
20190625_cell-08.czi
20190625_cell-09.czi
20190625_cell-10.czi
20190625_cell-11.czi
20190625_cell-12.czi
20190625_cell-13.czi
20190625_cell-14.czi
20190625_cell-20.czi
20190625_cell-23.czi
20190625_cell-25.czi
20190625_cell-26.czi
20190625_cell-27.czi
20190625_cell-31.czi
20190625_cell-32.czi
20190625_cell-33.czi
20190625_cell-34.czi
20190625_cell-35.czi
20190625_cell-36.czi
20190625_cellde-00.czi.ims
20190625_cellde-01.czi.ims
20190625_cellde-02.czi.ims
20190625_cellde-03.czi.ims
20190625_cellde-05.czi.ims
20190625_cellde-06.czi.ims
20190625_cellde-07.czi.ims
20190625_cellde-08.czi.ims
20190625_cellde-09.czi.ims
20190625_cellde-10.czi.ims
20190625_cellde-11.czi.ims
20190625_cellde-12.czi.ims
20190625_cellde-13.czi.ims
20190625_cellde-14.czi.ims
20190625_cellde-20.czi.ims
20190625_cellde-23.czi.ims
20190625_cellde-25.czi.ims
20190625_cellde-26.czi.ims
20190625_c

In [31]:
cellData.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,file,original cell name,Amy_mod,movie,x,y,ab1 (heterochromatin),...,random_10perc_ch1,hetChrom_10perc_ch2,random_10perc_ch2,hetChrom_10perc_ch3,random_10perc_ch3,exp_folder,hetChrom_newctl_volume,hetChrom_newctl_ch1,hetChrom_newctl_ch2,hetChrom_newctl_ch3
0,0,0,0,20190625_cell-00.czi,7LE1(B),1130/10,190619-cdc6-dhb-pcna002xy09,172,956,HP1B,...,281528813.0,174067475.0,131640512.0,1.352231e+09,1.043517e+09,20190625-MSL rename,280370,5.739116e+09,3.726297e+09,9.123801e+09
1,1,1,1,20190625_cell-01.czi,7L-E1(T),0,190619-cdc6-dhb-pcna002xy09,158,906,HP1B,...,266516067.0,162411790.0,128299406.0,1.353023e+09,1.065184e+09,20190625-MSL rename,250606,5.403750e+09,3.628343e+09,8.997863e+09
2,2,2,2,20190625_cell-02.czi,7L-E2(B),0,190619-cdc6-dhb-pcna002xy09,832,252,HP1B,...,318905411.0,194622718.0,148176740.0,1.366670e+09,1.118365e+09,20190625-MSL rename,189925,6.729417e+09,4.442309e+09,1.076495e+10
3,3,3,3,20190625_cell-03.czi,7L-E2(T),0,190619-cdc6-dhb-pcna002xy09,888,160,HP1B,...,285348983.0,195376713.0,139563277.0,1.298882e+09,1.038559e+09,20190625-MSL rename,220076,5.459073e+09,3.733122e+09,9.257708e+09
4,4,4,4,20190625_cell-05.czi,7L-G2,0,190619-cdc6-dhb-pcna002xy09,172,324,HP1B,...,698512214.0,248463390.0,183210463.0,2.583220e+09,2.158853e+09,20190625-MSL rename,436430,8.007695e+09,4.024894e+09,1.674243e+10


In [32]:
cellData.to_csv(cellDataFile.replace('20_50_10_box','20_50_10_newctrl'))